In [2]:
from roblox_ml.data import DataConnector
import numpy as np
import json
import datasets
import os
import shutil

/home/jovyan/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
# alternative route: make a table of all the universe ids
from roblox_ml.artifacts import ArtifactClient
import pandas as pd
import numpy as np
bdir = '/home/jovyan/data/nhirschkind/big_icons2/train'
fnames = os.listdir(bdir)
uids = [int(eval(x.split('.')[0])) for x in fnames]

parquet = pd.DataFrame({'universe_id': uids})
parquet['universe_id'] = parquet['universe_id'].astype(np.int64)
parquet.to_parquet('/home/jovyan/data/nhirschkind/temp.pq', index=False, compression='snappy')
cli = ArtifactClient()

cli.upload_file_to_s3('/home/jovyan/data/nhirschkind/temp.pq', 's3://ml-platform-generic/nhirschkind/thumbnails_v1/fuafua.parquet')


2023-12-15T18:57:37.920 INFO artifact_client - upload_file_to_s3: File uploaded - /home/jovyan/data/nhirschkind/temp.pq -> s3://ml-platform-generic/nhirschkind/thumbnails_v1/fuafua.parquet


In [17]:
parquet.head()

,universe_id
0,2721131192
1,3503969316
2,5109712929
3,5075216487
4,4959610056


In [18]:
# now add the table to hive

add_query = """
CREATE EXTERNAL TABLE IF NOT EXISTS usr.nhirschkind_thumbnails_v1
(universe_id BIGINT
)-- these fields should match your panda dataframe schema
STORED AS PARQUET
LOCATION 's3://ml-platform-generic/nhirschkind/thumbnails_v1'
"""

dc = DataConnector()
dc.hive_query_to_dataframe(add_query)

/tmp/ipykernel_5506/3999120307.py:12: DeprecatedWarning: hive_query_to_dataframe is deprecated. Use the "sparkconnector" class instead
  dc.hive_query_to_dataframe(add_query)
/home/jovyan/.local/lib/python3.10/site-packages/thrift/transport/TSSLSocket.py:53: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  self._context = ssl.SSLContext(ssl_version)
/home/jovyan/.local/lib/python3.10/site-packages/roblox_ml/data/dataconnector.py:99: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, self._hive_connection)


TypeError: 'NoneType' object is not iterable

In [23]:
universe_detail_query = """
    SELECT
        t1.universe_id AS universe_id,
        t2.name AS name,
        t2.description AS description
    FROM usr.nhirschkind_thumbnails_v1 t1 LEFT JOIN
    sqlserver.robloxuniverses_universes t2 ON
    t1.universe_id=t2.id AND t2.ds='2023-12-01'
        """

dc = DataConnector()
result = dc.trino_query_to_dataframe(universe_detail_query)

In [24]:
len(result)

359326

In [30]:
from tqdm import tqdm
dictform = {}
for i, row in tqdm(result.iterrows()):
    dictform[row['universe_id']] = (row['name'], row['description'])

prompt_template = 'The thumbnail of a Roblox game called "{name}"'
metadata = []

for uid in tqdm(uids):
    name, description = dictform[uid]
    if not isinstance(name, str):
        continue
    if description is None:
        description = "(No description available)"
    #name = name.replace('\n',' ')
    #description = description.replace('\n', ' ')
    prompt = prompt_template.format(name=name, description=description)
    metadata.append({'file_name': str(uid) + '.png', 'text': prompt})


359326it [00:12, 28491.71it/s]
100%|██████████| 359326/359326 [00:00<00:00, 534143.79it/s]


In [31]:
import json
with open('/home/jovyan/data/nhirschkind/big_icons2/train/metadata.jsonl', 'w') as f:
    for entry in metadata:
        json.dump(entry, f)
        f.write('\n')

In [ ]:
# test all images
from PIL import Image
import shutil
fnames = os.listdir('/home/jovyan/data/nhirschkind/big_icons2/train/')
for fname in tqdm(fnames):
    try:
        Image.open(os.path.join('/home/jovyan/data/nhirschkind/big_icons2/train/', fname))
    except Exception as e:
        print('failure')
        os.remove(os.path.join('/home/jovyan/data/nhirschkind/big_icons2/train/', fname))
    

  1%|▏         | 5250/359327 [00:11<13:11, 447.60it/s]

failure


  6%|▌         | 19772/359327 [02:04<50:05, 112.98it/s]

failure


  7%|▋         | 24476/359327 [02:42<42:05, 132.59it/s]